# Mapbox Stitcher

This notebook contains tools to fetch tiles from mapbox, in a user-defined style, and stitch them together into 1 big map image

### Usage

1) Add a line in the file "regions.csv" like the examples with the parameters you want. For the correspondence between zoomlevel and scale, see: https://wiki.openstreetmap.org/wiki/Zoom_levels

2) fill in your mapbox acces token and username (username is only needed for custom styles)

3) in REGION_NAME: fille in the name you have given the region in the csv file

4) go to the section "Print out image info", klick on the code cell. Then click in the topbar on "run", click "run all obove selected cell", hit ctrl+enter. This wil print out information about the map to be generated. your available RAM/10 is a save upper bound for the size of the map that will be generated.

5) If the info is as expected, go to "run"->"run selected cell and below". This will fetch the tiles from the server and construct the map. Large maps can take while to download and stitch together!

! watch out for warnings about tilesize, a rong tilesize leads to wrong metadata and coordinate transformations

Timing: generating a 27x18 tiles image, corresponding to 13824x9216 pixels or 127.40 Mpixel, takes about a minute with decent internet on a intel 7th gen i5 laptop. 

Memory usage: ~1.5GB

Size output image: 54.1MB

## ToDo

This is also an oportunity to define tile resolution from user scope

display region(s) of interest

## Imports

In [1]:
# numpy for array manipulations
import numpy as np
# the workhorse, provides routines to download tiles and do some
# coordinate transforms
import cartopy.io.img_tiles as cartotiles
# image manipulating and saving
from PIL import Image
# polygons to define regions
from shapely.geometry.polygon import Polygon
# more coordinate support
import globalmaptiles as gmaptiles
# for working with metadata in csv
import pandas as pd
# for parallel downloads
import concurrent.futures
# for inline plotting
import matplotlib.pyplot as plt

## Global variables
This should be the only variables you need to change to generate a different map, unless you need more in-depth modifications. Changes to these maps should be handled as much as possible by changing the mapbox style, or doing postprocessing on the resulting image

In [14]:
# acces token for mapbox
TOKEN = "FILL IN ACCES TOKEN"
# username corresponding to token, only needed for private styles
USER = "FILL IN USERNAME"

# descriptive name for region of interest, used to fetch map info from regions.csv
REGION_NAME = "ExampleOutdoors"

# if you intend to print the map, set the DPI value to get the physical size of the image
DPI = 200
# size in pixels of 1 tile. Default is 256 for custom styles, unless you did "louche aanpassing" in the sourcecode of cartopy, 
# and for satellite data the default is 256 as well. Otherwise, use 512
TILESIZE = 512

# you shouldn't need to change these ;)
CM_PER_INCH = 2.54
EARTH_CIRCUMFERENCE = 40_075_016.686   # meter
SEP = ";"

In [3]:
regionfile = pd.read_csv("regions.csv", sep=SEP)
regionfile = regionfile.set_index("Region")

MAP_PARAMS = regionfile.loc[REGION_NAME]
ZOOM = int(MAP_PARAMS["ZoomLevel"])
LON = (MAP_PARAMS["LonMin"], MAP_PARAMS["LonMax"])
LAT = (MAP_PARAMS["LatMin"], MAP_PARAMS["LatMax"])
STYLE_NAME = MAP_PARAMS["StyleName"]
STYLE_ID = MAP_PARAMS["StyleID"]
CUSTOM_TILESET = MAP_PARAMS["CustomTileset"]
GOOGLE_TILES = MAP_PARAMS["GoogleTiles"]  # useful for satellite data, mapbox satellite tends to 404 when not used in custom style
FOLDER = MAP_PARAMS["Folder"]

## local helper functions

In [4]:
def tile_coords(lon, lat, zoom):
    """ 
    Converts a lat, lon coordinate to the (x,y,z) coordinates
    of the tile containing lat, lon at the given zoom level
    for calculations, see: https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames
    
    Parameters
    ----------
    lon (float):
        The longitude of the point in degrees (decimal)
    lat (float):
        The latitude of the point in degrees (decimal), between -85.0511 and +85.0511
    zoom (int):
        The zoom level, should be between 0 and 22 for mapbox, 0 and 18 for Openstreetmap
        
    Returns
    -------
    slipmapcoordinates (Tuple):
        Tuple (x,y,z) of ints corresponding to the tile coordinates on a slipmap at zoom level z
    """
    # conversion factors
    DEG_TO_RAD = np.pi/180
    n = 2**zoom
    
    xtile = n * ((lon+180)/360)
    ytile = n * (1 - (np.log(np.tan(lat*DEG_TO_RAD) + 1/np.cos(lat*DEG_TO_RAD))/np.pi))/2
    
    return (int(np.floor(xtile)), int(np.floor(ytile)), zoom)

def area_tiles(lon1, lat1, lon2, lat2, zoom):
    """ 
    Calculates the (x,y,z) coordinates of all tiles overlapping with the rectangle
    definide by (lon1,lat1), (lon2,lat2) at zoom level zoom
    
    Parameters
    lon1, lon2 (float):
        longitude of coordinates defining rectangle in degrees
    lat1, lat2 (float):
        latitiude of coordinates defining rectangle in degrees
    zoom (int):
        zoomlevel
    """
    lon_min = min(lon1,lon2)
    lon_max = max(lon1,lon2)
    lat_min = min(lat1,lat2)
    lat_max = max(lat1,lat2)
    
    # get tile coordinates for (min,min) and (max,max)
    # coordinates start from north pole!
    x_min, y_min, z = tile_coords(lon_min, lat_max, zoom)
    x_max, y_max, z = tile_coords(lon_max, lat_min, zoom)
    
    # construct all tile coords and collect them in matrix
    x_coords = np.arange(x_min, x_max+1, 1)
    y_coords = np.arange(y_min, y_max+1, 1)
    
    return (x_coords, y_coords)

def download_tiles(x_coords, y_coords, zoom):
    """ 
    Downloads tiles in bbox determined by area_tiles
    """
    # fetch all tiles
    tiles = []
    tile_imgs = []
    print("rows: %d" %len(y_coords))
    print("tiles per row: %d" %len(x_coords))
    for y_coord in y_coords:
        row = []
        row_imgs = []
        print("row: %d" %(y_coord-y_coords[0]+1))
        for x_coord in x_coords:
            tile = maptiles.get_image((x_coord, y_coord, zoom))
            tile_img = tile[0]
            row.append(tile)
            row_imgs.append(tile_img)
        tiles.append(row)
        tile_imgs.append(row_imgs)
        
    return (tiles, tile_imgs)
    
def stitch_image(tile_imgs):
    """ 
    stitches tiles downloaded by download_tiles together into 1 big map
    """
    # stitch image together
    rows = len(tile_imgs)
    columns = len(tile_imgs[0])
    tile_size = tile_imgs[0][0].size[0]

    # create new, empty, image
    big_img = Image.new("RGB", (columns*tile_size, rows*tile_size))

    # fill in the image
    for row in range(rows):
        for column in range(columns):
            big_img.paste(tile_imgs[row][column], box=(column*tile_size, row*tile_size))
    
    return big_img

In [5]:
# fetches all tiles for a given domain

# copied from sourcefile because the image stitcher stitches tiles together wrongly, mapboxtiles have NO overlap

def images_for_domain(maptilesinstance, target_domain, target_z):
    tiles = []

    def fetch_tile(tile):
        try:
            img, extent, origin = maptilesinstance.get_image(tile)
        except OSError:
            # Some services 404 for tiles that aren't supposed to be
            # there (e.g. out of range).
            raise
        img = np.array(img)
        #x = np.linspace(extent[0], extent[1], img.shape[1])
        #y = np.linspace(extent[2], extent[3], img.shape[0])
        return img, tile, origin

    with concurrent.futures.ThreadPoolExecutor(
            max_workers=maptiles._MAX_THREADS) as executor:
        futures = []
        for tile in maptilesinstance.find_images(target_domain, target_z):
            futures.append(executor.submit(fetch_tile, tile))
        for future in concurrent.futures.as_completed(futures):
            try:
                img, tile, origin = future.result()
                tiles.append([img, tile, origin])
            except OSError:
                pass

    return tiles

# function to merge the tiles recieved from images_for_domain
def merge_tiles(tiles, image_pixel_size, area_tile_coords):
    img = np.zeros((image_pixel_size[1], image_pixel_size[0], 3),np.uint8)
    origin = (area_tile_coords[0][0], area_tile_coords[1][0])

    for tile in tiles:
        # get the tilecoords in the map:
        mtc = (tile[1][0]-origin[0], tile[1][1]-origin[1])
        # get the pixel coords of the upperleft pixel
        ptc = (mtc[0]*TILESIZE, mtc[1]*TILESIZE)
        #print(ptc)
        # fill in the image, array is transpose of xy-coords for maptiles!
        img[ptc[1]:ptc[1]+TILESIZE,ptc[0]:ptc[0]+TILESIZE] = tile[0]
    
    return img

## calculate extra global variable from user input

In [6]:
# instance to download tiles
if CUSTOM_TILESET:
    maptiles = cartotiles.MapboxStyleTiles(TOKEN, USER, STYLE_ID, cache=False)
elif GOOGLE_TILES:
    # google also provide the styles "street", "terrain" and "only_street", see: https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.io.img_tiles.GoogleTiles.html#cartopy.io.img_tiles.GoogleTiles
    maptiles = cartotiles.GoogleTiles(style=STYLE_ID)
else:
    maptiles = cartotiles.MapboxTiles(TOKEN, STYLE_ID)
    
# coordinate transform instance
map_coords = gmaptiles.GlobalMercator(tileSize=512)
# constant
PIXEL_SCALE_EQUATOR = EARTH_CIRCUMFERENCE/(2**ZOOM*TILESIZE)  # meter/pixel

# construct polygon of area
corners = [(LON[0], LAT[0]), (LON[0], LAT[1]), (LON[1], LAT[1]), (LON[1], LAT[0])]
area = Polygon([ (np.mean(maptiles.tile_bbox(*tile_coords(*corner, ZOOM))[0]), np.mean(maptiles.tile_bbox(*tile_coords(*corner, ZOOM))[1]) ) for corner in corners])
# find all tiles intersecting the polygon
area_tile_coords = area_tiles(LON[0], LAT[0], LON[1], LAT[1], ZOOM)
# set the origin of 

# get extent of map in different coordinate systems
# extend in tile-ids
extent_tile_id = [area_tile_coords[0][0], area_tile_coords[0][-1],
                  area_tile_coords[1][0], area_tile_coords[1][-1]]

# extent in pseudo-mercator coordinates
extent1 = maptiles.tile_bbox(*tile_coords(LON[0],LAT[0],ZOOM))
extent2 = maptiles.tile_bbox(*tile_coords(LON[1],LAT[1],ZOOM))
meterx = np.concatenate([extent1[0], extent2[0]])
metery = np.concatenate([extent1[1], extent2[1]])

extent_pseudo_mercator = (min(meterx), max(meterx),
                          min(metery), max(metery))
del extent1, extent2, meterx, metery

# extent in lat-lon degrees
lat_min, lon_min = map_coords.MetersToLatLon(extent_pseudo_mercator[0], extent_pseudo_mercator[2])
lat_max, lon_max = map_coords.MetersToLatLon(extent_pseudo_mercator[1], extent_pseudo_mercator[3])
extent_lat_lon = (lat_min, lat_max, lon_min, lon_max)
del lat_min, lon_min, lat_max, lon_max

# extent in pixels
min_point = (map_coords.MetersToPixels(extent_pseudo_mercator[0], extent_pseudo_mercator[2], ZOOM))
max_point = (map_coords.MetersToPixels(extent_pseudo_mercator[1], extent_pseudo_mercator[3], ZOOM))

map_pixel_extent = (int(min_point[0]), int(max_point[0]), int(min_point[1]), int(max_point[1]))
map_pixel_origin = (int(min_point[0]), int(min_point[1]))
# cleanup
del min_point, max_point

calculate image info to show to user

In [7]:
# precalculate the size of the resulting image in pixels
image_pixel_size = (len(area_tile_coords[0])*TILESIZE,
                    len(area_tile_coords[1])*TILESIZE)

# calculate the scale of the image (top and bottom, in meter/pixel)
pixel_scale_bottom = np.cos(LAT[0]*np.pi/180)*PIXEL_SCALE_EQUATOR
pixel_scale_top = np.cos(LAT[1]*np.pi/180)*PIXEL_SCALE_EQUATOR

# calculate the physical size of the image
image_phys_size = (image_pixel_size[0]/DPI*CM_PER_INCH,image_pixel_size[1]/DPI*CM_PER_INCH)

# calculate how much larger the actual geographical area displayed is
point0 = map_coords.LatLonToMeters(LAT[0],LON[0])
point1 = map_coords.LatLonToMeters(LAT[1],LON[1])
# area of region of interest in square meters (map coordinates)
meter_area_of_interest = (point1[0]-point0[0])*(point1[1]-point0[1])
# actual area covered by map (square meters, map coordinates)
meter_area_mapped = (extent_pseudo_mercator[1]-extent_pseudo_mercator[0])*\
                    (extent_pseudo_mercator[3]-extent_pseudo_mercator[2])
# calculate ratio
surplus_area = 1-meter_area_of_interest/meter_area_mapped
del point0, point1

## Print out image info

In [8]:
description = "------ SIZE ------\n\n"
description += """Size of resulting image will be (in pixels): 
width: %d
height: %d
yielding a %.2f Megapixel image\n\n""" %(image_pixel_size[0],image_pixel_size[1],
                                     image_pixel_size[0]*image_pixel_size[1]/10**6)

description += """When printed in a resolution of %.1f DPI, the physical size of this print in cm is:
width: %.2f
height: %.2f
yielding a %.3f m^2 print\n\n""" %(DPI,image_phys_size[0],image_phys_size[1],
                                   image_phys_size[0]*image_phys_size[1]/10**4)
description += """The scale of the map in meter/pixel is:
top: %.3f 
bottom: %.3f\n\n""" %(pixel_scale_top, pixel_scale_bottom)

description += "-----COVERAGE-------\n\n"
description += """The actual geographical area covered ranges is between:
lat: %.6f° - %.6f°
lon: %.6f° - %.6f°\n""" %extent_lat_lon
description += "This leads to a map that is %.2f%s bigger than the region of interest" %(100*surplus_area,"%")
print(description)

------ SIZE ------

Size of resulting image will be (in pixels): 
width: 3584
height: 3072
yielding a 11.01 Megapixel image

When printed in a resolution of 200.0 DPI, the physical size of this print in cm is:
width: 45.52
height: 39.01
yielding a 0.178 m^2 print

The scale of the map in meter/pixel is:
top: 6.467 
bottom: 6.484

-----COVERAGE-------

The actual geographical area covered ranges is between:
lat: 47.249407° - 47.428087°
lon: 8.437500° - 8.745117°
This leads to a map that is 34.65% bigger than the region of interest


## download and save the image

In [9]:
# download the tiles
tiles = images_for_domain(maptiles, area, ZOOM)
# fix tilesize in case it was wrong
actual_tilesize = np.shape(tiles[0][0])[0]
if actual_tilesize != TILESIZE:
    print("WARNING: TILESIZE defined in General Parameters does not match tilesize recieved from server.")
    print("actual tilesize is: %d"%actual_tilesize)
    print("consider running the notebook again with the right tilesize to generate correct metadata")
    TILESIZE = actual_tilesize

In [10]:
img = merge_tiles(tiles, image_pixel_size, area_tile_coords)

In [12]:
PIL_image = Image.fromarray(img)
PIL_image.save("%s/%s_%s_%s.png"%(FOLDER, REGION_NAME, STYLE_NAME, ZOOM), "png")

## write map metadata to csv

In [13]:
COLUMNS = ["MapName", "Region", "StyleName", "StyleID", "IsPrivateStyle", "user", 
           "LatMin", "LatMax", "LonMin", "LonMax", "Zoom", "TileSize", 
           "PixWidth", "PixHeight", "PixScaleTop", "PixScaleBottom",
           "MeterXMin", "MeterXMax", "MeterYMin", "MeterYMax",
           "MapLatMin", "MapLatMax", "MapLonMin", "MapLonMax",
           "MapPixXMin", "MapPixXMax", "MapPixYMin", "MapPixYMax",
           "SurplusArea"]
MAPNAME = "%s_%s_%s"%(REGION_NAME, STYLE_NAME, ZOOM)

# try to open the csv in case it already exists
try: 
    df = pd.read_csv("maps_metadata.csv", sep=SEP)
except:
    # csv doesn't exist, create new dataframe
    df = pd.DataFrame(columns=COLUMNS)
df = df.set_index("MapName")

# check if this map is already in the csv
if MAPNAME in df.index.values:
    # drop the row
    df = df.drop(MAPNAME)

# create data array
data = {"MapName": MAPNAME,
        "Region" : REGION_NAME, 
        "StyleName" : STYLE_NAME, 
        "StyleID" : STYLE_ID, 
        "IsPrivateStyle" : CUSTOM_TILESET, 
        "user" : USER, 
        "LatMin" : LAT[0], 
        "LatMax" : LAT[1], 
        "LonMin" : LON[0], 
        "LonMax" : LON[1], 
        "Zoom" : ZOOM, 
        "TileSize" : TILESIZE, 
        "PixWidth" : image_pixel_size[0], 
        "PixHeight" : image_pixel_size[1], 
        "PixScaleTop" : pixel_scale_top, 
        "PixScaleBottom" : pixel_scale_bottom,
        "MeterXMin" : extent_pseudo_mercator[0], 
        "MeterXMax" : extent_pseudo_mercator[1], 
        "MeterYMin" : extent_pseudo_mercator[2], 
        "MeterYMax" : extent_pseudo_mercator[3],
        "MapLatMin" : extent_lat_lon[0], 
        "MapLatMax" : extent_lat_lon[1], 
        "MapLonMin" : extent_lat_lon[2], 
        "MapLonMax" : extent_lat_lon[3],
        "MapPixXMin" : map_pixel_extent[0], 
        "MapPixXMax" : map_pixel_extent[1], 
        "MapPixYMin" : map_pixel_extent[2], 
        "MapPixYMax" : map_pixel_extent[3],
        "SurplusArea" : surplus_area}#, 
        #"description" : description}

# append data array
#df = df.append(data, ignore_index=True)
df.loc[data["MapName"]] = [data[key] for key in COLUMNS[1:]]

# write out csv
df.to_csv("maps_metadata.csv", sep=SEP)